In [95]:
#simple RNN modle
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle
import numpy as np
import time

In [96]:
with open('./data/data.pickle', 'rb') as f:
    data = pickle.load(f)

In [97]:
with open('./data/words.pickle', 'rb') as f:
    words = pickle.load(f)

In [98]:
word_vec_init = {}
for i, j in enumerate(words):
    word_vec_init[j] = torch.zeros(4809, 1, dtype=torch.float)
    word_vec_init[j][i, 0] = 1.

word_vec_init['other'] = torch.zeros(4809, 1, dtype=torch.float)
word_vec_init[j][4808, 0] = 1.

In [99]:
class RNN_1(nn.Module):
    def __init__(self, words_vec):
        super(RNN_1, self).__init__()
        self.words_vec = words_vec
        self.matrix1 = torch.rand(4809, 4809, requires_grad=True, dtype=torch.float)
        self.matrix2 = torch.rand(4809, 4809, requires_grad=True, dtype=torch.float)
        self.matrix3 = torch.rand(4809, 4809, requires_grad=True, dtype=torch.float)
        self.b1 = torch.zeros(4809, 1, requires_grad=True, dtype=torch.float)
        self.b2 = torch.zeros(4809, 1, requires_grad=True, dtype=torch.float)
        self.b3 = torch.zeros(4809, 1, requires_grad=True, dtype=torch.float)
        
        self.allparameters = [self.matrix1, self.matrix2, self.matrix3, self.b1, self.b2, self.b3]
        
    def forward(self, x, m1=None, m2=None):
        for i, j in self.words_vec.items():
            if x == i:
                x0 = j
                break
            
            if i == 'other':
                #print('這個是other')
                x0 = j
        
        t = 0
        if t == 0:
            t = 1
            y1 = (self.matrix1.mm(x0) + self.b1) / 1000
            y2 = (self.matrix2.mm(y1) + self.b2) / 1000
            y3 = (self.matrix3.mm(y2) +  self.b3) / 1000
        
        else:
            if m1 & m2:
                y1 = (self.matrix1.mm(x0) + self.b1 + m1) / 1000
                y2 = (self.matrix2.mm(y1) + self.b2 + m2) / 1000
                y3 = (self.matrix3.mm(y2) +  self.b3) / 1000
                
            else:
                y1 = (self.matrix1.mm(x0) + self.b1) / 1000
                y2 = (self.matrix2.mm(y1) + self.b2) / 1000
                y3 = (self.matrix3.mm(y2) + self.b3) / 1000
        cc = ((y1 + y2) / 2).detach()
        self.words_vec[x] = cc.clone()
        del(cc)
        
        return y1, y2, y3

In [100]:
class loss_fn(nn.Module):
    def __init__(self, words_vec):
        super(loss_fn, self).__init__()
        self.words_vec = words_vec
        
    def forward(self, pred, target_word=None):
        if target_word == None:
            return None
        
        for i, j in self.words_vec.items():
            if target_word == i:
                target = j
                break
            
            if i == 'other':
                #print('這個是other')
                target = j
        
        loss = torch.zeros(4809, 1, dtype=torch.float)
        for i in range(pred.size()[0]):
            loss[i, 0] = pred[i, 0] - target[i, 0]
        loss = loss.norm(p=2, dim=0, keepdim=True)    
        
        return loss

In [104]:
def learn(data, model, loss_func, optimizer):
    for i1, i2 in enumerate(data):
        print('第  {}  筆資料----------------------'.format(i1))
        for j1, j2 in enumerate(i2):
            print('第  {}  句話'.format(j1))
            m1 = None
            m2 = None
            for k1, k2 in enumerate(j2):
                #print('第  {}  個字'.format(k1))
                m1, m2, m3 = model(k2, m1=m1, m2=m2)
                if k1 < len(j2)-1:
                    try:
                        loss = loss_func(m3, j2[k1+1])
                        time.sleep(1)
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
                    except:
                        print('錯誤')

In [105]:
rnn = RNN_1(word_vec_init)
loss_func = loss_fn(word_vec_init)
optimizer = torch.optim.SGD(rnn.allparameters, lr=1e-5)

In [106]:
data0 = []
for i, j in data.items():
    data0.append(j[1])

In [ ]:
learn(data0, rnn, loss_func, optimizer)

第  0  筆資料----------------------
第  0  句話
第  1  句話
第  2  句話
第  3  句話
第  4  句話
錯誤
第  5  句話
第  6  句話
錯誤
錯誤
錯誤
錯誤
第  7  句話


In [78]:
data0[0][0]

'你有做超音波嘛，那我們來看報告，有些部分有紅字耶。'

In [90]:
rnn.matrix3

tensor([[0.6585, 0.6632, 0.0649,  ..., 0.5977, 0.4874, 0.2572],
        [0.3243, 0.1278, 0.0160,  ..., 0.3772, 0.4391, 0.1952],
        [0.8113, 0.2271, 0.0245,  ..., 0.6491, 0.9799, 0.3763],
        ...,
        [0.8314, 0.9963, 0.6432,  ..., 0.3259, 0.0041, 0.5155],
        [0.4378, 0.1108, 0.4247,  ..., 0.4296, 0.6539, 0.9203],
        [0.6746, 0.9344, 0.3977,  ..., 0.8595, 0.4479, 0.7637]],
       requires_grad=True)

In [80]:
rnn.b2.grad

In [81]:
word_vec_init['報']

tensor([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]])

In [82]:
s = data[0][1][0]
m1 = m2 = None
i = 0
s

'你有做超音波嘛，那我們來看報告，有些部分有紅字耶。'

In [83]:
m2

In [91]:
word_vec_init[s[0]]

tensor([[0.0008],
        [0.0006],
        [0.0009],
        ...,
        [0.0007],
        [0.0007],
        [0.0008]])

In [92]:
y1, y2, y3 = rnn(s[0], None, None)
y3

tensor([[0.0118],
        [0.0119],
        [0.0118],
        ...,
        [0.0118],
        [0.0119],
        [0.0118]], grad_fn=<DivBackward0>)

In [93]:
loss = loss_func(y3, s[15])
loss

tensor([[0.8215]], grad_fn=<NormBackward1>)

In [94]:
loss.backward()

In [85]:
loss_func.retain_graph

ModuleAttributeError: 'loss_fn' object has no attribute 'retain_graph'

In [187]:
y1, y2, y3 = rnn(s[i], m1, m2)
m1 = y1
m2 = y2
loss = loss_func(y3, s[i+1])
optimizer.zero_grad()
loss.backward()
optimizer.step()
i += 1
print(loss)

RuntimeError: Trying to backward through the graph a second time, but the saved intermediate results have already been freed. Specify retain_graph=True when calling backward the first time.

In [178]:
rnn.b3

tensor([[-4.7858e-10],
        [-4.8181e-10],
        [-4.8815e-10],
        ...,
        [-4.9376e-10],
        [-4.8811e-10],
        [-4.8762e-10]], requires_grad=True)

In [181]:
word_vec_init['告']

tensor([[0.0007],
        [0.0008],
        [0.0008],
        ...,
        [0.0010],
        [0.0006],
        [0.0006]], grad_fn=<DivBackward0>)

In [62]:
#測試
y = x.mm(x)
y

tensor([[0.0015, 0.2359],
        [0.0047, 0.8872]], grad_fn=<MmBackward>)

In [32]:
torch.tensor([[1, 2, 3, 4], [1, 2, 3, 4], [1, 2, 3, 4], [1, 2, 3, 4]])

tensor([[1, 2, 3, 4],
        [1, 2, 3, 4],
        [1, 2, 3, 4],
        [1, 2, 3, 4]])

In [79]:
x = torch.tensor([[4], [3]], dtype=torch.float)

In [65]:
x.norm(p=2, dim=0, keepdim=True)

tensor([[5.]])

In [80]:
x

tensor([[4.],
        [3.]])

In [81]:
(x + x)/2

tensor([[4.],
        [3.]])

In [121]:
time.sleep(3)

In [64]:
x = torch.tensor([[4, 5]], requires_grad=True, dtype=torch.float)

In [65]:
y = torch.zeros(1, 2, dtype=torch.float)

In [66]:
y.requires_grad

False

In [67]:
x.requires_grad

True

In [68]:
y = x.detach().clone()

In [72]:
y

tensor([[4., 5.]])

In [70]:
y.requires_grad

False

In [71]:
x[0, 1] = 6.0